**Table of contents**<a id='toc0_'></a>    
- [Общие методы](#toc1_)    
  - [Обработка дат](#toc1_1_)    
  - [Обработка уровней орагнизации](#toc1_2_)    
  - [Обработка КБК](#toc1_3_)    
  - [Поиск и замена плохих адресов](#toc1_4_)    
  - [Декомпозиция дат](#toc1_5_)    
- [Обработка ORG](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [12]:
import os
import re
import datetime
import copy

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from tqdm.auto import tqdm
from pandarallel import pandarallel
from pullenti.address.AddressService import AddressService

AddressService.set_server_connection("http://localhost:2222")

tqdm.pandas()
pandarallel.initialize(progress_bar=False)
# Сброс ограничений на число столбцов
pd.set_option("display.max_columns", 200)

# Сброс ограничений на количество символов в записи
pd.set_option("display.max_colwidth", 200)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [13]:
path_dir = '../data/raw_data/contract/2014/'
for file_name in tqdm(os.listdir(path_dir)):
       path_file = os.path.join(path_dir, file_name)
       df = pd.read_csv(path_file, dtype='str', sep='|')
       df.columns = ['number_contract', 'address_customer', 'full_name_customer',
              'short_name_customer', 'code', 'code_type', 'id_customer',
              'inn_customer', 'kpp_customer', 'code_form_org', 'okpo_code',
              'municipal_code', 'budget_name', 'extrabudget_name', 'budget_level',
              'contract_status', 'notice', 'ikz_code', 'id_contract_electronic',
              'unique_number_plan', 'method_determinig_supplier', 'date_summarizing',
              'date_posting', 'grouds_single_supplier', 'document_details',
              'info_support', 'find_date_contract', 'date_performance',
              'date_contract_registry', 'date_update_registry',
              'date_start_performance', 'date_end_performance', 'contract_item',
              'contract_price', 'contract_price_nds', 'prepayment_amount',
              'performance_security', 'size_performance_quality', 'warranty_period',
              'place_performance', 'full_name_supplier', 'inn_supplier',
              'kpp_supplier', 'code_okpo_supplier', 'date_registration_supplier',
              'country_supplier', 'code_country_supplier', 'address_supplier',
              'postal_address_supplier', 'contact', 'status_supplier', 'kbk']
       df.to_csv(path_file, sep='|', index=False)


  0%|          | 0/180 [00:00<?, ?it/s]

100%|██████████| 180/180 [12:54<00:00,  4.30s/it]


In [15]:
path_dir = '../data/contract_number/split_data/contract/2014/'
for file_name in tqdm(os.listdir(path_dir)):
       path_file = os.path.join(path_dir, file_name)
       df = pd.read_csv(path_file, dtype='str', sep='|')
       df.columns = ['number_contract', 'address_customer', 'inn_customer']
       df.to_csv(path_file, sep='|', index=False)

100%|██████████| 500/500 [02:12<00:00,  3.78it/s]


# <a id='toc1_'></a>[Общие методы](#toc0_)

## <a id='toc1_1_'></a>[Обработка дат](#toc0_)

In [7]:
dict_month = {
    "января": "01",
    "февраля": "02",
    "марта": "03",
    "апреля": "04",
    "мая": "05",
    "июня": "06",
    "июля": "07",
    "августа": "08",
    "сентября": "09",
    "октября": "10",
    "ноября": "11",
    "декабря": "12",
}


def date_extract(date: str):
    if not date or date == "--.--.----":
        return None

    date = date.replace("Загрузка ...", "").strip()
    try:
        return datetime.datetime.strptime(date, "%d.%m.%Y").date()
    except ValueError:
        pass

    try:
        return datetime.datetime.strptime(date[:10], "%d.%m.%Y").date()
    except ValueError:
        pass

    try:
        return datetime.datetime.strptime(date.split()[0], "%d.%m.%Y").date()
    except ValueError:
        pass

    for key, value in dict_month.items():
        if key in date:
            date = date.replace(key, value)
            date = ".".join(date.split())

    date = date.split(".")

    if len(date) == 2:
        date = ".".join(["01"] + date)
        return datetime.datetime.strptime(date[:10], "%d.%m.%Y").date()
    # логи

## <a id='toc1_2_'></a>[Обработка уровней орагнизации](#toc0_)

In [8]:
list_local = ["муниципальный уровень", "местный бюджет"]
list_sub = [
    "уровень субъекта рф",
    "бюджет субъекта российской федерации",
    "бюджет территориального государственного внебюджетного фонда",
    "бюджет территориального государственного внебюджетного фонда",
]
list_fed = [
    "федеральный уровень",
    "федеральный бюджет",
    "бюджет пенсионного фонда российской федерации",
    "бюджет федерального фонда обязательного медицинского страхования",
    "бюджет фонда социального страхования российской федерации",
]

list_fed_2 = [
    "войскавая",
    "войсковая",
    "воениз",
    "федеральн",
    "район водных путей и судоходства",
    "следвест",
    "пенсион",
    "росган",
    "фгбу",
    "всероссийс",
    " фбу",
    "прокурату",
    "университ",
    "научно-исследователь",
    "государственное научное учреждение",
    "росграниц",
    "российской академии наук",
    "внутренних дел",
    "мвд",
    "суд",
    "управление министерства промышленности",
    "торговли российской федерации",
    "таможенный пост",
    "российской федерации",
    "таможня",
    "таможенного",
]

list_local_2 = [
    "городская администрация",
    "муниципал",
    "школа",
    "детский сад",
    "городского поселения",
    "городского округа",
    "администрация рабочего поселка",
    "совет",
    "управление образованием администрации г",
    "поселок",
    "поселк",
    "частное учереждение",
    "администрации зато",
    "администрация пгт",
    "территориальная избирательная комиссия",
]
list_sub_2 = [
    "центр занятости населения",
    "област",
    "республи",
    "края",
    "край",
    "краев",
    "города",
    "автоном",
    "oбластное",
    "здравоохране",
    "больниц",
    "родильный дом",
    "профессиональн",
    "детский дом",
    "дом-интернат",
    "социальн",
    "поликлиник",
    "больниц",
    "государственное бюджетное общеобразовательное учреждение",
    "государственное бюджетное образовательное учреждение",
    "медико-санитарная часть",
    "московское государственное унитарное предприятие",
    "учреждение культуры города",
    "государственное бюджетное учреждение культуры",
    "центр социального обслуживания",
    "социального обслуживания граждан",
    "стоматологическая поликлиника",
    "центр для детей-сирот и детей",
    "санкт-петербур",
    "фонд социального страхования российской федерации",
    "государственное бюджетное учреждение",
    "инспекция труда",
    "региональный",
]
list_anothe = [
    "акционерное общество",
    "завод",
    "акционерное московское общество",
    "общество с ограниченной ответственностью",
    "частное",
]
list_fed_3 = []
list_sub_3 = [
    "ветеринар",
]
list_local_3 = ["района", "сельск"]

inn_sub = {
    "7727795994": 'Государственное бюджетное научное учреждение "Московский институт развития образования"',
    "4205050521": 'государственное учреждение "Кузбасспассажиравтотранс"',
    "1001036026": 'БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "ЦЕНТР КУЛЬТУРНЫХ ИНИЦИАТИВ" (АГЕНТСТВО "КУЛЬТУРНАЯ СЕТЬ КАРЕЛИИ")',
}
inn_fed = {"7704193182": "Региональное оперативно-поисковое управление"}
inn_mun = {
    "3509009509": 'БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "КОММУНАЛЬЩИК"',
    "3511005766": 'БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ КУЛЬТУРЫ "КИРИЛЛОВСКИЙ КИНОДОСУГОВЫЙ ЦЕНТР"',
}
inn_another = {"1633002328": 'ДЕТСКИЙ ОЗДОРОВИТЕЛЬНЫЙ ЛАГЕРЬ "ЧАЙКА"'}

name_for_result = ["местный", "субъектовый", "федеральный", "иное"]


def fillna_budget_level(budget_level: str, full_name_customer: str, inn_customer: str):
    if budget_level:
        for list_name, name in zip(
            [list_local, list_sub, list_fed], ["местный", "субъектовый", "федеральный"]
        ):
            for name_trigger in list_name:
                if name_trigger.lower() in budget_level:
                    return name

    # если не получилось выделить данные из budget_level попробуем сделать это с full_name_customer
    if full_name_customer:
        for list_name, name in zip(
            [list_fed_2, list_local_2, list_sub_2, list_anothe],
            ["федеральный", "местный", "субъектовый", "иное"],
        ):
            for name_trigger in list_name:
                if name_trigger.lower() in full_name_customer:
                    return name

        for list_name, name in zip(
            [list_fed_3, list_local_3, list_sub_3],
            ["федеральный", "местный", "субъектовый", "иное"],
        ):
            for name_trigger in list_name:
                if name_trigger.lower() in full_name_customer:
                    return name

        if (
            "администрац" in full_name_customer
            or "комитет по управлению имуществом" in full_name_customer
        ) and not all(
            [
                i in full_name_customer
                for i in ["моксв", "севастопол" "президент", "санкт-петербур"]
            ]
        ):
            return "местный"

        if "городская дума" in full_name_customer and "моксв" not in full_name_customer:
            return "местный"

    if inn_customer:
        for inn_dict, name in zip(
            [inn_mun, inn_sub, inn_fed, inn_another],
            ["местный", "субъектовый", "федеральный", "иное"],
        ):
            for inn in inn_dict.keys():
                if inn == inn_customer:
                    return name
    # добавить логги

    return None

## <a id='toc1_3_'></a>[Обработка КБК](#toc0_)

In [9]:
kbk_type = pd.read_excel("../data/kbk.xlsx", sheet_name="type", dtype="str")
kbk_np = pd.read_excel("../data/kbk.xlsx", sheet_name="np", dtype="str")
kbk_section = pd.read_excel("../data/kbk.xlsx", sheet_name="section", dtype="str")

In [10]:
def extract_data_from_kbk(kbk, year):
    dict_kbk = {
        "code_main_admin": None,
        "code_section_sub": None,
        "code_direction_expenses": None,
        "code_type_expenses": None,
        "code_national_project": None,
        "value_code_section": None,
        "value_code_sub": None,
        "value_code_type_expenses": None,
        "name_national_project": None,
        "name_fed_national_project": None,
    }
    if not kbk:
        return dict_kbk

    if len(kbk) == 3 or kbk[:-3] == "0" * 17:
        code_type_expenses = kbk[-3:]
        value_code_type_expenses = kbk_type.loc[
            kbk_type.code == code_type_expenses, "mean"
        ].to_list()

        if len(value_code_type_expenses):
            dict_kbk["value_code_type_expenses"] = value_code_type_expenses[0]
        else:
            pass
            # логи
        dict_kbk["code_type_expenses"] = code_type_expenses
        return dict_kbk

    elif len(kbk) == 20:
        kbk_search = re.compile(r"(\S\S\S)(\S\S\S\S)(\S\S\S\S\S\S\S\S\S\S)(\S\S\S)")
        kbk_find = kbk_search.search(kbk)

        # код главного распоредителя бюджетных средств
        code_main_admin = kbk_find.group(1)
        # print('Код ГРС:', code_main_admin)
        # код раздела и подраздела
        code_section_sub = kbk_find.group(2)
        # print('Код раздела и подраздела:', code_section_sub)
        # код целевой статьи
        code_direction_expenses = kbk_find.group(3)
        # print('Код целевой статьи:', code_direction_expenses)
        # код вида расходов
        code_type_expenses = kbk_find.group(4)
        # print('Код вида расходов:', code_type_expenses)
        # код национального проекта
        code_national_project = (
            code_direction_expenses[3:5] if not code_direction_expenses[3].isdigit() else None
        )
        # print('Код национального проекта:', code_national_project)

        value_code_section = kbk_section.loc[
            (kbk_section.year == year) & (kbk_section.code == code_section_sub[:2]), "mean"
        ].to_list()
        if len(value_code_section):
            dict_kbk["value_code_section"] = value_code_section[0]
        else:
            pass
        # print('value_code_section:', value_code_section)

        value_code_sub = kbk_section.loc[
            (kbk_section.year == year) & (kbk_section.code == code_section_sub), "mean"
        ].to_list()
        if len(value_code_sub):
            dict_kbk["value_code_sub"] = value_code_sub[0]
        else:
            pass
        # print('code_type_expenses:', value_code_sub)

        value_code_type_expenses = kbk_type.loc[
            kbk_type.code == code_type_expenses, "mean"
        ].to_list()
        if len(value_code_type_expenses):
            dict_kbk["value_code_type_expenses"] = value_code_type_expenses[0]
        else:
            pass

        # print('value_code_type_expenses:', value_code_type_expenses)
        if code_national_project:
            list_national_project = kbk_np.loc[
                (kbk_np.year == year) & (kbk_np.code == code_national_project),
                ["name_national_project", "name_fed_national_project"],
            ].values
            # print('list_national_project:', list_national_project)

            if len(list_national_project):
                dict_kbk["name_national_project"] = list_national_project[0]
                dict_kbk["name_fed_national_project"] = list_national_project[1]
            else:
                pass
                # логи

        dict_kbk["code_main_admin"] = code_main_admin
        dict_kbk["code_section_sub"] = code_section_sub
        dict_kbk["code_direction_expenses"] = code_direction_expenses
        dict_kbk["code_type_expenses"] = code_type_expenses
        dict_kbk["code_national_project"] = code_national_project

        return dict_kbk

    else:
        # print(kbk)
        return dict_kbk
        # добавить логи

## <a id='toc1_4_'></a>[Поиск и замена плохих адресов](#toc0_)

In [11]:
data_org = pd.dataFrame()


def check_address(address, code, code_type):
    list_check = ["Российская Федерация", "РФ", "обл", "ул", "край", "г,", "п."]
    is_nan = type(address) == float
    is_telephon = address.replace("-", "").isdigit()
    is_email = ("@" in address) and not any([i in address for i in list_check])

    need_replace = any([is_nan, is_telephon, is_email])

    if not need_replace:
        return address
    else:
        address = data_org.loc[
            (data_org.code == code) & (data_org.code_type == code_type), "address"
        ]
        return address

## <a id='toc1_5_'></a>[Декомпозиция дат](#toc0_)

In [80]:
class DecompositionAddress:
    def __init__(self, path_for_cache: str):
        self.path_for_cache = path_for_cache
        self.columns = [
            "country",
            "regioncity",
            "regionarea",
            "district",
            "settlement",
            "city",
            "citydistrict",
            "locality",
            "territory",
            "street",
            "plot",
            "building",
            "apartment",
            "room",
            "coef",
        ]
        if not os.path.exists(path_for_cache):
            pd.DataFrame(columns=["address"] + self.columns).to_csv(
                path_for_cache, sep="|", index=False
            )

        self.dict_cahce = pd.read_csv(
            path_for_cache, sep="|", dtype="str", index_col="address"
        ).to_dict(orient="index")

    def address_decompose(self, address: str):
        if not address:
            return {key: None for key in self.columns}

        if address in self.dict_cahce:
            return self.dict_cahce[address]

        else:
            return self.use_pullenti(address)

    def use_pullenti(self, address: str):
        dict_res = {key: None for key in self.columns}
        process_address = AddressService.process_single_address_text(address)

        dict_res["coef"] = process_address.coef

        for address_element in process_address.items:
            level = str(address_element.level).split(".")[1].lower()
            element_address = address_element.to_string_min()
            dict_res[level] = element_address

        self.add_address_to_cache(address, dict_res)
        dict_res.pop("coef")
        return dict_res

    def add_address_to_cache(self, address, dict_result):
        self.dict_cahce[address] = dict_result
        dict_result_for_df = dict_result.copy()
        dict_result_for_df["address"] = address
        pd.DataFrame(dict_result_for_df, index=[0])[["address"] + self.columns].to_csv(
            self.path_for_cache, sep="|", index=False, mode="a", header=False
        )

In [81]:
address_dec = DecompositionAddress(path_for_cache="../data/cache/cache_address.csv")

In [82]:
for i in tqdm(df.address.unique()):
    address_dec.address_decompose(i)

100%|██████████| 4488/4488 [00:00<00:00, 595377.06it/s]


In [83]:
process_address = AddressService.process_single_address_text(
    "Российская Федерация, 678280, Саха /Якутия/ Респ, Сунтарский у, Сарданга с, УЛ. СЕМЕНА СЕМЕНОВА, Д.43"
)

In [84]:
for address_element in process_address.items:
    print("level", address_element.level)
    print(address_element.to_string_min())
print("coef:", process_address.coef)

level AddrLevel.LOCALITY
село Сарданг
level AddrLevel.STREET
улица Семенова
level AddrLevel.BUILDING
д.43
coef: 30


In [85]:
process_address.coef

30

# <a id='toc2_'></a>[Обработка ORG](#toc0_)

In [94]:
df = pd.read_csv("../data/raw_data/org/2014_1/0.csv", sep="|", dtype="str")
print(df.columns)
df.shape

Index(['code', 'code_type', 'access_blocking', 'full_name', 'short_name',
       'adress', 'code_registr', 'date_registration', 'date_last_change',
       'inn', 'kpp', 'ogrn', 'oktmo', 'location', 'iky', 'date_iky',
       'code_okfs', 'name_property', 'okpf_code', 'okopf_name', 'credentials',
       'date_registration_tax', 'organization_type', 'organization_level',
       'okpo_code', 'okfd_code', 'budget_code', 'budget_name', 'telephone',
       'fax', 'postal_adress', 'email', 'site', 'contact_person', 'time_zone'],
      dtype='object')


(4499, 35)

In [95]:
address_dec = DecompositionAddress(path_for_cache="../data/cache/cache_aderess.csv")


def processing_date_org(df: Series, columns: list):
    dict_result = {}
    # если мы получает nan при обращении к дате в DataFrame, то
    # np.nan == nan возращает False, однако у nan type float, когда у всех дат str
    inn = df["inn"]
    # unique_code = df["code"] + df['code_type']

    date_registration = df["date_registration"] if type(df["date_registration"]) == str else None
    date_last_change = df["date_last_change"] if type(df["date_last_change"]) == str else None
    date_registration_tax = (
        df["date_registration_tax"] if type(df["date_registration_tax"]) == str else None
    )
    date_iky = df["date_iky"] if type(df["date_iky"]) == str else None

    dict_result["date_registration"] = date_extract(date_registration)
    dict_result["date_last_change"] = date_extract(date_last_change)
    dict_result["date_registration_tax"] = date_extract(date_registration_tax)
    dict_result["date_iky"] = date_extract(date_iky)

    # определяем уровень бюджета
    organization_level = (
        str(df["organization_level"]).lower() if type(df["organization_level"]) == str else None
    )
    full_name_customer = str(df["full_name"]).lower()
    inn = str(df["inn"])

    dict_result["organization_level"] = fillna_budget_level(
        organization_level, full_name_customer, inn
    )

    # раскладываем даты
    dict_address = address_dec.address_decompose(df["address"])

    dict_result.update(dict_address)

    return pd.Series(dict_result)[columns].to_list()

In [96]:
columns = [
    "date_registration",
    "date_last_change",
    "date_registration_tax",
    "date_iky",
    "organization_level",
    "country",
    "regioncity",
    "regionarea",
    "district",
    "settlement",
    "city",
    "citydistrict",
    "locality",
    "territory",
    "street",
    "plot",
    "building",
    "apartment",
    "room",
]
df_copy = df.copy()

df_copy[columns] = df_copy.apply(
    lambda x: processing_date_org(x, columns=columns), axis=1, result_type="expand"
)

In [97]:
df_copy[columns].isnull().sum()

date_registration           4
date_last_change          255
date_registration_tax       5
date_iky                 4499
organization_level          0
country                    49
regioncity               4223
regionarea                326
district                 2407
settlement               4445
city                     2021
citydistrict             4488
locality                 2780
territory                4406
street                    222
plot                     4491
building                  153
apartment                4380
room                     4497
dtype: int64

In [98]:
df_copy.loc[df_copy.regionarea.isnull(), ["address", "regioncity"]]

,adress,regioncity
3,"Российская Федерация, 199053, Санкт-Петербург, ПРОСПЕКТ СРЕДНИЙ В.О., ДОМ 20/ЛИТЕРА А",город Санкт-Петербург
17,"Российская Федерация, 678240, Саха /Якутия/ Респ, Верхневилюйский у, Дюллюкю с, УЛИЦА П.МИХАЙЛОВА, 9",NaN
23,"Российская Федерация, 678290, Саха /Якутия/ Респ, Сунтарский у, Кемпендяй с, ПЕР. ШКОЛЬНЫЙ, Д.9",NaN
67,"Российская Федерация, 195197, Санкт-Петербург, УЛ. ЗАМШИНА, Д. 42/ЛИТЕР А",город Санкт-Петербург
95,"Российская Федерация, 125375, Москва, ПЕР ДЕГТЯРНЫЙ, Д. 6/СТР. 1",город Москва
...,...,...
4422,"Российская Федерация, 196244, Санкт-Петербург, ПР-КТ ВИТЕБСКИЙ, Д.33/К.6, ЛИТЕРА А",город Санкт-Петербург
4425,"Российская Федерация, 109316, Москва, УЛ. МЕЛЬНИКОВА, Д. 4/СТР. 2",город Москва
4433,"Российская Федерация, 198096, Санкт-Петербург, УЛ. АВТОВСКАЯ, Д.11/ЛИТЕР А",город Санкт-Петербург
4449,"Российская Федерация, 678774, Саха /Якутия/ Респ, Верхнеколымский у, Верхнеколымск с, УЛ НАБЕРЕЖНАЯ, 5",NaN


In [100]:
path_test = "../data/raw_data/org/2014_1/"
prob = []
num_problem = 0
df_buffer = pd.DataFrame(columns=["organization_level", "full_name", "inn"])
for file_name in tqdm(sorted(os.listdir(path_test), key=lambda x: int(x.removesuffix(".csv")))):
    df_test_now = pd.read_csv(os.path.join(path_test, file_name), sep="|", dtype="str")
    df_test_now[columns] = df_test_now.parallel_apply(
        lambda x: processing_date_org(x, columns=columns), axis=1, result_type="expand"
    )
    problem_level_org = df_test_now.organization_level.isnull().sum()
    num_problem += problem_level_org
    if problem_level_org:
        prob.append((file_name, problem_level_org))
        df_buffer = pd.concat(
            [
                df_buffer,
                df_test_now.loc[
                    df_test_now.organization_level.isnull(),
                    ["organization_level", "full_name", "inn"],
                ],
            ]
        )
num_problem

0

In [102]:
df_copy.columns

Index(['code', 'code_type', 'access_blocking', 'full_name', 'short_name',
       'adress', 'code_registr', 'date_registration', 'date_last_change',
       'inn', 'kpp', 'ogrn', 'oktmo', 'location', 'iky', 'date_iky',
       'code_okfs', 'name_property', 'okpf_code', 'okopf_name', 'credentials',
       'date_registration_tax', 'organization_type', 'organization_level',
       'okpo_code', 'okfd_code', 'budget_code', 'budget_name', 'telephone',
       'fax', 'postal_adress', 'email', 'site', 'contact_person', 'time_zone',
       'country', 'regioncity', 'regionarea', 'district', 'settlement', 'city',
       'citydistrict', 'locality', 'territory', 'street', 'plot', 'building',
       'apartment', 'room'],
      dtype='object')

In [103]:
class ProcessingData:
    def __init__(self, path_cache_address: str, path_cache_org_address:str):
        self.columns = [
            "code",
            "code_type",
            "access_blocking",
            "full_name",
            "short_name",
            "address",
            "code_registr",
            "date_registration",
            "date_last_change",
            "inn",
            "kpp",
            "ogrn",
            "oktmo",
            "location",
            "iky",
            "date_iky",
            "code_okfs",
            "name_property",
            "okpf_code",
            "okopf_name",
            "credentials",
            "date_registration_tax",
            "organization_type",
            "organization_level",
            "okpo_code",
            "okfd_code",
            "budget_code",
            "budget_name",
            "telephone",
            "fax",
            "postal_address",
            "email",
            "site",
            "contact_person",
            "time_zone",
            "country",
            "regioncity",
            "regionarea",
            "district",
            "settlement",
            "city",
            "citydistrict",
            "locality",
            "territory",
            "street",
            "plot",
            "building",
            "apartment",
            "room",
        ]
        self.address_dec = DecompositionAddress(path_for_cache=path_cache_address)
        
        if not os.path.exists(path_cache_org_address):
            pd.DataFrame(columns=['unique', 'address']).to_csv(path_cache_org_address, sep='|', index=False)
            
        df_cahce = pd.read_csv(path_cache_org_address, sep='|', index='unique')
        df_cahce = df_cahce.set_index('unique')
        
        self.cache_org_address = df_cahce.to_dict(orient="index")
        


    def date_formatting(self, date: str):
        if not date or (date == "--.--.----" or date == "--.--.--"):
            return None

        date = date.replace("Загрузка ...", "").strip()
        try:
            return datetime.datetime.strptime(date, "%d.%m.%Y").date()
        except ValueError:
            pass

        try:
            return datetime.datetime.strptime(date[:10], "%d.%m.%Y").date()
        except ValueError:
            pass

        try:
            return datetime.datetime.strptime(date.split()[0], "%d.%m.%Y").date()
        except ValueError:
            pass

        for key, value in dict_month.items():
            if key in date:
                date = date.replace(key, value)
                date = ".".join(date.split())

        date = date.split(".")

        if len(date) == 2:
            date = ".".join(["01"] + date)
            return datetime.datetime.strptime(date[:10], "%d.%m.%Y").date()

    def fillna_org_level(self, org_level: str, full_name_customer: str, inn_customer: str):
        if org_level:
            for list_name, name in zip(
                [list_local, list_sub, list_fed], ["местный", "субъектовый", "федеральный"]
            ):
                for name_trigger in list_name:
                    if name_trigger.lower() in org_level:
                        return name

        # если не получилось выделить данные из budget_level попробуем сделать это с full_name_customer
        if full_name_customer:
            for list_name, name in zip(
                [list_fed_2, list_local_2, list_sub_2, list_anothe],
                ["федеральный", "местный", "субъектовый", "иное"],
            ):
                for name_trigger in list_name:
                    if name_trigger.lower() in full_name_customer:
                        return name

            for list_name, name in zip(
                [list_fed_3, list_local_3, list_sub_3],
                ["федеральный", "местный", "субъектовый", "иное"],
            ):
                for name_trigger in list_name:
                    if name_trigger.lower() in full_name_customer:
                        return name

            if (
                "администрац" in full_name_customer
                or "комитет по управлению имуществом" in full_name_customer
            ) and not all(
                [
                    i in full_name_customer
                    for i in ["моксв", "севастопол" "президент", "санкт-петербур"]
                ]
            ):
                return "местный"

            if "городская дума" in full_name_customer and "моксв" not in full_name_customer:
                return "местный"

        if inn_customer:
            for inn_dict, name in zip(
                [inn_mun, inn_sub, inn_fed, inn_another],
                ["местный", "субъектовый", "федеральный", "иное"],
            ):
                for inn in inn_dict.keys():
                    if inn == inn_customer:
                        return name
        # добавить логги

        return None

    def processing_df_date_org(self, df: Series, columns: list):
        dict_result = {}
        # если мы получает nan при обращении к дате в DataFrame, то
        # np.nan == nan возращает False, однако у nan type float, когда у всех дат str
        inn = df["inn"]
        # unique_code = df["code"] + df['code_type']

        date_registration = df["date_registration"] if type(df["date_registration"]) == str else None
        date_last_change = df["date_last_change"] if type(df["date_last_change"]) == str else None
        date_registration_tax = (
            df["date_registration_tax"] if type(df["date_registration_tax"]) == str else None
        )
        date_iky = df["date_iky"] if type(df["date_iky"]) == str else None

        dict_result["date_registration"] = date_extract(date_registration)
        dict_result["date_last_change"] = date_extract(date_last_change)
        dict_result["date_registration_tax"] = date_extract(date_registration_tax)
        dict_result["date_iky"] = date_extract(date_iky)

        # определяем уровень организации (местный, субъектовый, федеральный)
        organization_level = (
            str(df["organization_level"]).lower() if type(df["organization_level"]) == str else None
        )
        full_name_customer = str(df["full_name"]).lower()
        inn = str(df["inn"])

        dict_result["organization_level"] = fillna_budget_level(
            organization_level, full_name_customer, inn
        )

        # раскладываем даты
        dict_address = self.address_dec.address_decompose(df["address"])

        dict_result.update(dict_address)

        return pd.Series(dict_result)[columns].to_list()

    def run_org(self, path_df)



## Обработка contract